# AgentsVille Trip Planner - Project Assignment

In this project, you'll implement an AI system to help you plan a trip--to the wonderful city of AgentsVille!

Your AI system will demonstrate advanced LLM reasoning techniques including:

1. **Role-Based Prompting** - Your agent will act as a specialized travel planner
2. **Chain-of-Thought Reasoning** - Step-by-step planning of itineraries
3. **ReAct Prompting** - Thought → Action → Observation cycles
4. **Feedback Loops** - Self-evaluation using tools in the ReAct loop to find mistakes and improve plans

You'll simulate external API calls to gather weather data and activities. Then, process this information to create personalized travel itineraries based on interests and other constraints. Last, you'll implement a feedback loop to refine the plan.

Your task is to build a travel agent that can plan the perfect AgentsVille vacation!

## Project Instructions

Here are the steps you'll follow:

1. **Define Vacation Details**
    - Specify the trip duration, interests, and constraints.
    - Use Pydantic to structure and verify this information in a class called `VacationInfo`.
2. **Review Weather and Activity schedules**
    - Simulate API calls to gather weather data and available activities in bulk
    - Review the data manually to understand the available options
3. **The ItineraryAgent**
    - Implement an agent that generates a day-by-day itinerary based on the vacation details
    - The system prompt will guide the agent's reasoning through a step-by-step planning process to take travel preferences (e.g. destination, dates, interests) and generate a detailed day-by-day itinerary
    - Craft the components of the prompt (including the role, task/instructions, output format, examples, and context) to elicit the best possible itinerary in one LLM call
4. **Evaluating the Itinerary**
    - Evaluate the itinerary using a set of criteria to ensure a high-quality travel plan
        - For instance, does the itinerary match the city and the dates requested?
        - Or, is the total cost calulation accurate and is it within budget?
        - Or, does the agent hallucinate any activities that are not available?
        - Or, does the agent suggest activities that are not suitable for the weather? This specific evaluation function will require the use of an LLM to compare the event description against the weather data.
5. **Defining the Tools**
    - We will define four tools to assist the agent
        - **calculator_tool**: to accurately calculate costs
        - **get_activities_by_date_tool**: to retrieve activities for a specific date
        - **run_evals_tool**: to evaluate the itinerary against the criteria
        - **final_answer_tool**: to provide the final answer in a structured format
6. **The ItineraryRevisionAgent**
    - We will implement a second agent that revises the itinerary based on feedback using the ReAct THOUGHT → ACTION → OBSERVATION cycle.
        - The LLM will first generated a THOUGHT / ACTION message, which contains reasoning steps and a tool call invocation.
        - The Python code will parse the tool call and execute it, returning the result as a string to the LLM in an OBSERVATION message.
        - After this cycle repeats n number of times, the LLM will invoke the final_answer_tool to signal to the Python code to end the loop and return the final answer.
    - This agent will also **incorporate feedback on the initial itinerary** from the travelers to ensure the final plan has **at least 2 activities per day**. A new evaluation function using a powerful LLM will be created to check this user feedback.
    - The agent will use the tools above to refine the plan iteratively, checking the weather and available activities, and ensuring the itinerary meets all constraints.
7. **Something just for fun!**
    - To wrap things up we'll create a fun, narrative summary of the trip, highlighting the best activities and experiences!

## Initial Setup

Let's start with settin up our environment and defining the vacation details.

In [1]:
# Let's make sure our computer can find all the special tools we need!
# This is like telling your friend where to find your toys in your room.
import os
import sys

WORKSPACE_DIRECTORY = "/workspace"
if os.path.exists(WORKSPACE_DIRECTORY) and WORKSPACE_DIRECTORY not in sys.path:
    sys.path.append(WORKSPACE_DIRECTORY)
    print(f"Added {WORKSPACE_DIRECTORY} to the Python path")

In [2]:
# Let's get all the special tools our computer needs to help us plan our trip!
# This is like making sure we have all our art supplies before starting a big project.
%pip install -q json-repair==0.47.1 numexpr==2.11.0 openai==1.74.0 pandas==2.3.0 pydantic==2.11.7 python-dotenv==1.1.0


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Now we need to connect to our AI friend who will help us plan our trip!
# This is like setting up a video call with a friend who lives far away.
from openai import OpenAI

client = OpenAI(
    # Change the base_url when using the Vocareum API endpoint
    # If using the OpenAI API endpoint, you can comment out the base_url line
    base_url="https://openai.vocareum.com/v1",
    # Uncomment one of the following
    api_key="voc-10171367811607362001419686ccf95120e31.64409803",  # <--- Using Vocareum API key
    # api_key=os.getenv(
    #     "OPENAI_API_KEY"
    # ),  # <-- Alternately, set as an environment variable
)


In [4]:
# Let's choose which AI brain to talk to - they're like different friends with different skills!
# We'll use GPT-4.1-mini because it's quick to respond and doesn't cost too much to talk to.
from enum import Enum


class OpenAIModel(str, Enum):
    GPT_41 = "gpt-4.1"  # Strong default choice for development tasks, particularly those requiring speed, responsiveness, and general-purpose reasoning. 
    GPT_41_MINI = "gpt-4.1-mini"  # Fast and affordable, good for brainstorming, drafting, and tasks that don't require the full power of GPT-4.1.
    GPT_41_NANO = "gpt-4.1-nano"  # The fastest and cheapest model, suitable for lightweight tasks, high-frequency usage, and edge computing.


MODEL = OpenAIModel.GPT_41_MINI  # Default model for this project


## Define Vacation Details

Let's encode the details of our vacation in JSON format and verify it using Pydantic.

In practice, a chatbot agent could gather the information of a user. After it has gathered all the information it needs, it could generate this JSON object from the chat transcript. We will skip that step to focus on the itinerary generation itself.

In [5]:
# Let's make a list of all the people going on our trip and what they like to do!
# This is like filling out a form before a vacation - who's coming, when, and what they enjoy.

VACATION_INFO_DICT = {
    "travelers": [
        {
            "name": "Yuri",
            "age": 30,
            # Possible interests: art, cooking, comedy, dancing, fitness, gardening, hiking, movies,
            # music, photography, reading, sports, technology, theatre, tennis, writing
            "interests": ["tennis", "cooking", "comedy", "technology"],
        },
        {
            "name": "Hiro",
            "age": 25,
            # Possible interests: art, cooking, comedy, dancing, fitness, gardening, hiking, movies,
            # music, photography, reading, sports, technology, theatre, tennis, writing
            "interests": ["reading", "music", "theatre", "art"],
        },
    ],
    "destination": "AgentsVille",
    "date_of_arrival": "2025-06-10",  # Mock data exists for 2025-06-10
    "date_of_departure": "2025-06-12",  # ...until 2025-06-15.
    "budget": 300,  # Budget is in fictional currency units.
}

In [6]:
# Let's make sure our vacation information is organized correctly!
# This is like having a special checklist to make sure we didn't forget anything important.

from project_lib import Interest  # This tells us what kinds of activities people might like
from typing import List  # This helps us make lists of things
from pydantic import BaseModel  # This helps us create organized information containers
import datetime  # This helps us work with dates
from pprint import pprint  # This helps us print information in a neat way


class Traveler(BaseModel):
    """A traveler with a name, age, and list of interests.

    Attributes:
        name (str): The name of the traveler.
        age (int): The age of the traveler.
        interests (List[Interest]): A list of interests of the traveler.
    """
    name: str
    age: int
    interests: List[Interest]


class VacationInfo(BaseModel):
    """Vacation information including travelers, destination, dates, and budget.
    Attributes:
        travelers (List[Traveler]): A list of travelers.
        destination (str): The vacation destination.
        date_of_arrival (datetime.date): The date of arrival.
        date_of_departure (datetime.date): The date of departure.
        budget (int): The budget for the vacation in fictional currency units.
    """
    travelers: List[Traveler]
    destination: str
    date_of_arrival: datetime.date
    date_of_departure: datetime.date
    budget: int


# Validate the VacationInfo data structure
vacation_info = VacationInfo.model_validate(VACATION_INFO_DICT)

# Display the vacation info as a dictionary
pprint(vacation_info.model_dump())

# Check that VacationInfo contains the expected data
assert "travelers" in vacation_info.model_dump().keys(), "VacationInfo should contain 'travelers' key"
assert "destination" in vacation_info.model_dump().keys(), "VacationInfo should contain 'destination' key"
assert "date_of_arrival" in vacation_info.model_dump().keys(), "VacationInfo should contain 'date_of_arrival' key"
assert "date_of_departure" in vacation_info.model_dump().keys(), "VacationInfo should contain 'date_of_departure' key"
assert "budget" in vacation_info.model_dump().keys(), "VacationInfo should contain 'budget' key"
assert isinstance(vacation_info.travelers, list), "Travelers should be a list"
assert all(isinstance(traveler, Traveler) for traveler in
           vacation_info.travelers), "All travelers should be instances of Traveler class"
assert isinstance(vacation_info.date_of_arrival, datetime.date), "date_of_arrival should be a date"
assert isinstance(vacation_info.date_of_departure, datetime.date), "date_of_departure should be a date"
assert isinstance(vacation_info.budget, int), "budget should be an integer"

# If all assertions pass, print a success message
print("✅ VacationInfo data structure is valid!")

{'budget': 300,
 'date_of_arrival': datetime.date(2025, 6, 10),
 'date_of_departure': datetime.date(2025, 6, 12),
 'destination': 'AgentsVille',
 'travelers': [{'age': 30,
                'interests': [tennis, cooking, comedy, technology],
                'name': 'Yuri'},
               {'age': 25,
                'interests': [reading, music, theatre, art],
                'name': 'Hiro'}]}
✅ VacationInfo data structure is valid!


## Review Weather and Activity Schedules

Now that we have the trip details, we can retrieve the weather and activity schedules for the dates of the trip.

We will call an API to get all the data at once, in order to be able to include it in the context for our itinerary planning agent.

Also, we will format the data as Pandas DataFrames for easier viewing. Take the time to read and understand the data to see if the agent notices the same things you do. For instance:
- What does the weather look like for the trip? On what days it is sunny, rainy, or cloudy?
- What activities are available on each day? Are there any special events or festivals related to the user's interests?

In [7]:
# Let's check what the weather will be like during our trip!
# This is like looking at a weather forecast app before packing your suitcase.

from project_lib import call_weather_api_mocked  # This helps us get weather information
import pandas as pd  # This helps us organize data in neat tables

pd.set_option("display.max_colwidth", None)  # This makes sure we can see all the weather details

weather_for_dates = [
    call_weather_api_mocked(
        date=ts.strftime("%Y-%m-%d"), city=vacation_info.destination
    )
    for ts in pd.date_range(
        start=vacation_info.date_of_arrival,
        end=vacation_info.date_of_departure,
        freq="D",
    )
]

weather_for_dates_df = pd.DataFrame(weather_for_dates)

weather_for_dates_df

,date,city,temperature,temperature_unit,condition,description
0,2025-06-10,AgentsVille,31,celsius,clear,A bright and sunny day in AgentsVille with clear skies and warm temperatures. Perfect weather for outdoor activities!
1,2025-06-11,AgentsVille,34,celsius,partly cloudy,"A warm day with periods of sunshine and mixed clouds, making it a perfect opportunity for outdoor activities."
2,2025-06-12,AgentsVille,28,celsius,thunderstorm,"A thunderstorm is expected to roll in during the afternoon, bringing heavy rain and gusty winds. The atmosphere will feel charged with humidity, creating a sultry and dramatic setting as clouds build in the sky."


In [8]:
# Let's find out what fun activities we can do in AgentsVille!
# This is like looking at a tourist guide to see what events are happening during our visit.

from project_lib import call_activities_api_mocked  # This helps us get information about available activities

activities_for_dates = [
    activity
    for ts in pd.date_range(
        start=vacation_info.date_of_arrival,
        end=vacation_info.date_of_departure,
        freq="D",
    )
    for activity in call_activities_api_mocked(
        date=ts.strftime("%Y-%m-%d"), city=vacation_info.destination
    )
]

activities_for_dates_df = pd.DataFrame(activities_for_dates)

activities_for_dates_df

,activity_id,name,start_time,end_time,location,description,price,related_interests
0,event-2025-06-10-0,FutureTech Breakfast Meet-Up,2025-06-10 09:00,2025-06-10 11:00,"The Innovation Atrium, Tech District, AgentsVille","Join fellow technology enthusiasts for a dynamic morning at the FutureTech Breakfast Meet-Up! Dive into the latest trends in tech, gadget demos, and networking opportunities over coffee and fresh pastries. Held indoors at the spacious Innovation Atrium, this event is perfect for tech lovers eager to exchange ideas and discover new possibilities in a comfortable, modern setting.",20,[technology]
1,event-2025-06-10-1,Serve & Savor: Tennis and Taste Luncheon,2025-06-10 12:00,2025-06-10 13:30,"The Grand Racquet Terrace, AgentsVille","Join us for 'Serve & Savor,' the ultimate crossover event for cooking and tennis enthusiasts in AgentsVille! Kick off your lunch hour with a friendly round of doubles on our outdoor courts, then unwind with a hands-on cooking workshop led by a local chef, where you'll prepare and enjoy delicious energy-boosting recipes. Whether you come for the sport or the flavors, this energizing luncheon celebrates both passions in a lively outdoor setting. Ideal for anyone who loves to play, cook, or simply savor fresh food and fun!",20,"[cooking, tennis]"
2,event-2025-06-10-2,Artful Athletics: Paint & Play Extravaganza,2025-06-10 15:00,2025-06-10 17:00,"Creative Courts Park, AgentsVille","Join us for an exciting afternoon at Creative Courts Park, where the worlds of art and sports collide! At 'Artful Athletics: Paint & Play Extravaganza', you'll participate in collaborative outdoor murals inspired by your favorite sports, and then get moving with fun, friendly sports mini-games. Whether you love painting or playing, this event celebrates creativity, teamwork, and the joy of movement under the open sky. Perfect for art lovers and sports enthusiasts alike—come ready to express yourself and get active! (Event is held outdoors; in case of rain, we move indoors to the Community Gym nearby.)",15,"[art, sports]"
3,event-2025-06-10-3,AgentsVille Twilight Writing Escape,2025-06-10 19:00,2025-06-10 21:00,"The Ink Loft, 12 Quill Lane, AgentsVille","Join fellow writers for an inspiring evening at The Ink Loft, where words flow as freely as the coffee! This writing-themed event welcomes all—novelists, poets, bloggers, or anyone with a passion for storytelling. Set indoors in AgentsVille's coziest lounge, enjoy writing games, group prompts, and opportunities to read your work aloud. Connect, create, and celebrate the art of writing in this creative indoor haven.",15,"[writing, reading, art]"
4,event-2025-06-11-0,Morning Groove Dance Party,2025-06-11 09:00,2025-06-11 10:30,"Rhythm Hall, Center Plaza, AgentsVille","Start your day with energy and joy at the Morning Groove Dance Party! This lively event welcomes dancers of all levels to join a vibrant indoor session filled with upbeat music and fun routines. Whether you love modern pop, Latin beats, or classic disco, our dance instructors will guide you to move and groove. Connect with fellow dance lovers in the colorful atmosphere of Rhythm Hall. Perfect for fans of dancing, music, and fitness. Let the rhythm move you! (Indoor event.)",15,"[dancing, music, fitness]"
5,event-2025-06-11-1,Tech Lunch & Learn: AI Frontiers,2025-06-11 12:00,2025-06-11 13:30,"The Digital Atrium, AgentsVille","Join fellow tech enthusiasts for a dynamic lunchtime event exploring the future of artificial intelligence! Held indoors at The Digital Atrium, this Tech Lunch & Learn features engaging lightning talks, interactive demos, and networking opportunities all centered around technology and innovation. Enjoy light lunch fare as you connect with others passionate about technology, AI, and the digital world. Whether you're a seasoned developer or just curious about tech, this event is for you! Related interests: technology, music (sound tech demos), photography (AI imaging), writing (AI cr

## The ItineraryAgent

First we will review the Pydantic objects used for defining the output of our agent, the TravelPlan, ItineraryDay, Activity, and Weather classes.

Second, we will create a Chain-of-Thought prompt to guide the agent in planning the trip. This prompt will instruct the agent to consider the weather, activities, and user preferences when creating the itinerary.

Third, we will run the agent to produce the TravelPlan object, which will will refine in the following steps.

In [9]:
# Let's create a special container to hold our vacation plan!
# This is like making a scrapbook with pages for each day of our trip.
# Each page will show the weather, fun activities, and why we chose them.
# Our AI helper will fill this scrapbook with a perfect plan for our vacation!

class Weather(BaseModel):
    temperature: float
    temperature_unit: str
    condition: str


class Activity(BaseModel):
    activity_id: str
    name: str
    start_time: datetime.datetime
    end_time: datetime.datetime
    location: str
    description: str
    price: int
    related_interests: List[Interest]


class ActivityRecommendation(BaseModel):
    activity: Activity
    reasons_for_recommendation: List[str]


class ItineraryDay(BaseModel):
    date: datetime.date
    weather: Weather
    activity_recommendations: List[ActivityRecommendation]


class TravelPlan(BaseModel):
    city: str
    start_date: datetime.date
    end_date: datetime.date
    total_cost: int
    itinerary_days: List[ItineraryDay]

In [10]:
# Let's tell our AI travel planner exactly what we want it to do!
# This is like giving instructions to a friend who's helping plan your vacation.
# We'll tell it to:
# 1. Think about what activities match our interests
# 2. Check the weather so we don't plan outdoor activities when it's raining
# 3. Make sure we don't spend more than our budget
# 4. Create a fun schedule for each day of our trip
# 5. Explain why it chose each activity for us

import json  # This helps us work with structured data
from project_lib import ChatAgent  # This is our AI helper that can talk to the big AI brain
from typing import Optional  # This helps us say that some information might be missing

ITINERARY_AGENT_SYSTEM_PROMPT = f"""
You are an expert Itinerary Planning Agent for AgentsVille. Your role is to create personalized travel plans that match travelers' interests while considering weather conditions and budget constraints.

## Task

Create a detailed day-by-day itinerary for travelers visiting AgentsVille based on the following guidelines:
1. Select activities that match the travelers' interests
2. Avoid outdoor-only activities during rainy or thunderstorm weather
3. Ensure the total cost stays within the specified budget
4. Include at least one activity per day
5. Distribute activities throughout the day (morning, afternoon, evening)
6. Consider the weather forecast when planning activities
7. Provide clear reasons for each activity recommendation

## Output Format

Respond using two sections (ANALYSIS AND FINAL OUTPUT) in the following format:

    ANALYSIS:
    1. WEATHER ASSESSMENT: Analyze the weather for each day of the trip
    2. TRAVELER PREFERENCES: Summarize the travelers' interests and constraints
    3. ACTIVITY SELECTION: For each day, explain your activity choices and how they match interests
    4. BUDGET CALCULATION: Show the cost breakdown and ensure it's within budget
    5. FINAL CHECKS: Verify that all constraints are satisfied


    FINAL OUTPUT:

    ```json
    {{
      "city": "AgentsVille",
      "start_date": "2025-06-10",
      "end_date": "2025-06-12",
      "total_cost": 100,
      "itinerary_days": [
        {{
          "date": "2025-06-10",
          "weather": {{
            "temperature": 31,
            "temperature_unit": "celsius",
            "condition": "clear"
          }},
          "activity_recommendations": [
            {{
              "activity": {{
                "activity_id": "event-2025-06-10-0",
                "name": "FutureTech Breakfast Meet-Up",
                "start_time": "2025-06-10T09:00:00",
                "end_time": "2025-06-10T11:00:00",
                "location": "The Innovation Atrium, Tech District, AgentsVille",
                "description": "Join fellow technology enthusiasts for a dynamic morning...",
                "price": 20,
                "related_interests": ["technology"]
              }},
              "reasons_for_recommendation": [
                "Matches Yuri's interest in technology",
                "Indoor activity suitable for morning"
              ]
            }}
          ]
        }}
      ]
    }}
    ```

## Context

Weather data for the trip:
${weather_for_dates_df.to_json(orient='records')}

Available activities for the trip:
${activities_for_dates_df.to_json(orient='records')}
"""  # noqa

assert "TASK" in ITINERARY_AGENT_SYSTEM_PROMPT.upper(), "❌ ITINERARY_AGENT_SYSTEM_PROMPT should contain a 'TASK' section"
assert "OUTPUT FORMAT" in ITINERARY_AGENT_SYSTEM_PROMPT.upper(), "❌ ITINERARY_AGENT_SYSTEM_PROMPT should contain a 'OUTPUT FORMAT' section"


class ItineraryAgent(ChatAgent):
    """An agent that plans itineraries based on vacation information, weather, and activities."""
    system_prompt = ITINERARY_AGENT_SYSTEM_PROMPT

    def get_itinerary(self, vacation_info: VacationInfo, model: Optional[OpenAIModel] = None) -> TravelPlan:
        """Generates a travel itinerary based on the provided vacation information."""
        from project_lib import print_in_box
        response = (self.chat(
            user_message=vacation_info.model_dump_json(indent=2),
            add_to_messages=False,
            model=model or self.model,
        ) or "").strip()

        print_in_box(response, "Raw Response")

        # Parse the response
        json_text = response.strip()

        if "```json" in json_text:
            json_text = json_text.split("```json")[1].split("```")[0].strip()

        try:
            travel_plan = TravelPlan.model_validate_json(json_text)
            return travel_plan
        except Exception as e:
            print("Error validating the following text as TravelPlan JSON:")
            print(json_text)
            raise


itinerary_agent = ItineraryAgent(client=client, model=MODEL)


╔══════════════════════════════════════════[ ItineraryAgent - System Prompt ]══════════════════════════════════════════╗
║ You are an expert Itinerary Planning Agent for AgentsVille. Your role is to create personalized travel plans that    ║
║ match travelers' interests while considering weather conditions and budget constraints.                              ║
║ ## Task                                                                                                              ║
║ Create a detailed day-by-day itinerary for travelers visiting AgentsVille based on the following guidelines:         ║
║ 1. Select activities that match the travelers' interests                                                             ║
║ 2. Avoid outdoor-only activities during rainy or thunderstorm weather                                                ║
║ 3. Ensure the total cost stays within the specified budget                                                           ║
║ 4. Include at least one activ

In [11]:
# Generate the travel itinerary
# No changes needed here, though you can change the model to a different one if you want.

travel_plan_1 = itinerary_agent.get_itinerary(
    vacation_info=vacation_info,
    model=MODEL,  # Optionally, you can change the model here
)

if travel_plan_1 is not None:
    print("✅ Initial itinerary generated successfully. Congratulations!")


╔═══════════════════════════════════════════[ ItineraryAgent - User Prompt ]═══════════════════════════════════════════╗
║ {                                                                                                                    ║
║   "travelers": [                                                                                                     ║
║     {                                                                                                                ║
║       "name": "Yuri",                                                                                                ║
║       "age": 30,                                                                                                     ║
║       "interests": [                                                                                                 ║
║         "tennis",                                                                                                    ║
║         "cooking",           

## Evaluating the Itinerary

We've successfully created an itinerary, but how do we know if it's any good?

Now we will create some evaluation functions (sometimes called evals) to help us determine the quality of the itinerary. We will not only want our final output to be of the highest quality possible initially, but we also want to give the chance for the LLM to reflect on its own output and make improvements at a second pass.

If the itinerary does not meet all the criteria specified here, no worries! Afterwards, we will implement a feedback loop that will allow the agent to improve its plan iteratively.

In [12]:
# Let's create some special tools to check if our vacation plan is good!
# This is like having a checklist to make sure we didn't forget anything important.

class AgentError(Exception):  # This helps us tell our AI when it makes a mistake
    pass


class EvaluationResults(BaseModel):
    success: bool
    failures: List[str]
    eval_functions: List[str]


def get_eval_results(vacation_info, final_output, eval_functions) -> EvaluationResults:
    """
    Evaluates the final output of the itinerary agent against a set of evaluation functions.
    Args:
        vacation_info (VacationInfo): The vacation information used to generate the itinerary.
        final_output (TravelPlan): The final output from the itinerary agent.
        eval_functions (List[callable]): A list of evaluation functions to apply.
    Returns:
        EvaluationResults: An object containing the success status, any failures, and the names of the evaluation functions used.
    """
    from project_lib import print_in_box
    if not isinstance(vacation_info, VacationInfo):
        raise ValueError("vacation_info must be an instance of VacationInfo")
    if not isinstance(final_output, TravelPlan):
        raise ValueError("final_output must be an instance of TravelPlan")
    if not isinstance(eval_functions, list) or not all(
            callable(fn) for fn in eval_functions
    ):
        raise ValueError("eval_functions must be a list of callable functions")
    eval_results = []
    for eval_fn in eval_functions:
        try:
            eval_fn(vacation_info, final_output)
        except AgentError as e:
            error_msg = str(e)
            print_in_box(error_msg, title="Evaluation Error")
            print("\n\n")

            eval_results.append(error_msg)
    return EvaluationResults(
        success=len(eval_results) == 0,
        failures=eval_results,
        eval_functions=[fn.__name__ for fn in eval_functions],
    )


In [13]:
# Let's check if our vacation dates are correct!
# This is like making sure we booked the hotel for the right days.

def eval_start_end_dates_match(vacation_info: VacationInfo, final_output: TravelPlan):
    """Verifies that the arrival and departure dates in vacation_info match the start and end dates in final_output.

    Args:
        vacation_info (dict): Contains the vacation details including arrival and departure dates
        final_output (dict): Contains the itinerary details including start and end dates

    Raises:
        AgentError: If either the arrival date doesn't match the start date or the departure date doesn't match the end date
    """
    if (
            vacation_info.date_of_arrival != final_output.start_date
            or vacation_info.date_of_departure != final_output.end_date
    ):
        raise AgentError(
            f"Dates do not match: {vacation_info.date_of_arrival} != {final_output.start_date} or {vacation_info.date_of_departure} != {final_output.end_date}"
        )

    if final_output.start_date > final_output.end_date:
        raise AgentError(
            f"Start date is after end date: {final_output.start_date} > {final_output.end_date}"
        )


get_eval_results(
    vacation_info=vacation_info,
    final_output=travel_plan_1,
    eval_functions=[eval_start_end_dates_match],
)

EvaluationResults(success=True, failures=[], eval_functions=['eval_start_end_dates_match'])

In [14]:
# Let's make sure we're not spending too much money on our trip!
# This is like checking your piggy bank before buying souvenirs.


def eval_total_cost_is_accurate(vacation_info: VacationInfo, final_output: TravelPlan):
    """Verifies that the total cost stated in final_output matches the sum of all activity prices.

    Args:
        vacation_info (dict): Contains the vacation details
        final_output (dict): Contains the itinerary details including activities with prices and total cost

    Raises:
        AgentError: If the calculated total cost doesn't match the stated total cost
    """
    actual_total_cost = 0

    for itinerary_day in final_output.itinerary_days:
        for activity_recommendation in itinerary_day.activity_recommendations:
            actual_total_cost += activity_recommendation.activity.price

    stated_total_cost = int(final_output.total_cost)

    if actual_total_cost != stated_total_cost:
        raise AgentError(
            f"Stated total cost does not match calculated total cost: {actual_total_cost} != {stated_total_cost}"
        )


def eval_total_cost_is_within_budget(vacation_info: VacationInfo, final_output: TravelPlan):
    """Verifies that the total cost stated in final_output is within the budget specified in vacation_info.

    Args:
        vacation_info (dict): Contains the vacation details including budget
        final_output (dict): Contains the itinerary details including total cost

    Raises:
        AgentError: If the total cost exceeds the budget
    """
    stated_total_cost = int(final_output.total_cost)
    if stated_total_cost > vacation_info.budget:
        raise AgentError(
            f"Total cost exceeds budget: {stated_total_cost} > {vacation_info.budget}"
        )


get_eval_results(
    vacation_info=vacation_info,
    final_output=travel_plan_1,
    eval_functions=[eval_total_cost_is_accurate, eval_total_cost_is_within_budget],
)



╔═════════════════════════════════════════════════[ Evaluation Error ]═════════════════════════════════════════════════╗
║ Stated total cost does not match calculated total cost: 108 != 216                                                   ║
╚══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╝





EvaluationResults(success=False, failures=['Stated total cost does not match calculated total cost: 108 != 216'], eval_functions=['eval_total_cost_is_accurate', 'eval_total_cost_is_within_budget'])

In [15]:
# Let's make sure our AI didn't make up any fake activities!
# This is like checking that a restaurant really exists before you plan to eat there.

def eval_itinerary_events_match_actual_events(
        vacation_info: VacationInfo, final_output: TravelPlan
):
    """Verifies that the events listed in the itinerary match the actual events

    Args:
        vacation_info (dict): Contains the vacation details including traveler information and their interests
        final_output (dict): Contains the itinerary details including daily activities

    Raises:
        AgentError: If any traveler has no matching activities or if one traveler has more than twice
                   the number of matching activities compared to another traveler
    """
    from project_lib import call_activity_by_id_api_mocked
    event_ids_not_matching = []
    event_ids_missing = []

    for itinerary_day in final_output.itinerary_days:
        for activity_recommendation in itinerary_day.activity_recommendations:
            event_id = activity_recommendation.activity.activity_id
            # Assuming get_event_by_id is a function that retrieves the event by its ID

            reference_event = call_activity_by_id_api_mocked(event_id)

            if reference_event is None:
                event_ids_missing.append(event_id)

            elif Activity(**reference_event) != activity_recommendation.activity:
                print(
                    "---\n"
                    f"Event ID {event_id} does not match the reference event:\n"
                    f"Reference Event: {reference_event}\n"
                    f"Activity Event: {activity_recommendation.activity.model_dump()}"
                )
                event_ids_not_matching.append(event_id)
            else:
                # The event matches, so we can continue
                pass

    if event_ids_missing or event_ids_not_matching:
        raise AgentError(
            f"Event IDs missing: {event_ids_missing}\nEvent IDs not matching: {event_ids_not_matching}"
        )


get_eval_results(
    vacation_info=vacation_info,
    final_output=travel_plan_1,
    eval_functions=[eval_itinerary_events_match_actual_events],
)


EvaluationResults(success=True, failures=[], eval_functions=['eval_itinerary_events_match_actual_events'])

In [16]:
# Let's make sure everyone gets to do things they enjoy!
# This is like making sure each friend gets to choose at least one activity during a group outing.

def eval_itinerary_satisfies_interests(
        vacation_info: VacationInfo, final_output: TravelPlan
):
    """Verifies that the itinerary includes activities matching each traveler's interests.

    This function checks that each traveler has at least one activity in the itinerary that matches their interests.

        Args:
        vacation_info (dict): Contains the vacation details including traveler information and their interests
        final_output (dict): Contains the itinerary details including daily activities

    Raises:
        AgentError: If any traveler has no matching activities or if one traveler has more than twice
                   the number of matching activities compared to another traveler
    """
    traveler_to_interests = {}
    traveler_to_interest_hit_counts = {}

    for traveler in vacation_info.travelers:
        traveler_to_interests[traveler.name] = traveler.interests
        traveler_to_interest_hit_counts[traveler.name] = 0

    for traveler_name, interests in traveler_to_interests.items():
        for itinerary_day in final_output.itinerary_days:
            for activity_recommendation in itinerary_day.activity_recommendations:
                # Check if the activity matches any of the traveler's interests
                matching_interests = set(traveler_to_interests[traveler_name]) & set(
                    activity_recommendation.activity.related_interests
                )

                if matching_interests:
                    traveler_to_interest_hit_counts[traveler_name] += 1
                    print(
                        f"✅ Traveler {traveler_name} has a match with interest {matching_interests} at {activity_recommendation.activity.name}"
                    )

    travelers_with_no_interest_hits = [
        traveler
        for traveler, interest_hit_count in traveler_to_interest_hit_counts.items()
        if interest_hit_count == 0
    ]

    # If any of the travelers have 0 matches, raise an error
    if travelers_with_no_interest_hits:
        raise AgentError(
            f"Travelers {travelers_with_no_interest_hits} has no matches with the itinerary."
        )


get_eval_results(
    vacation_info=vacation_info,
    final_output=travel_plan_1,
    eval_functions=[eval_itinerary_satisfies_interests],
)


✅ Traveler Yuri has a match with interest {technology} at FutureTech Breakfast Meet-Up
✅ Traveler Yuri has a match with interest {cooking} at Palette & Palate: Art Meets Cooking Experience
✅ Traveler Yuri has a match with interest {technology} at Tech & Film Fusion Night
✅ Traveler Hiro has a match with interest {art, reading} at AgentsVille Twilight Writing Escape
✅ Traveler Hiro has a match with interest {music} at Morning Groove Dance Party
✅ Traveler Hiro has a match with interest {art, music} at AgentsVille Art & Music Fusion Fest
✅ Traveler Hiro has a match with interest {art} at Palette & Palate: Art Meets Cooking Experience


EvaluationResults(success=True, failures=[], eval_functions=['eval_itinerary_satisfies_interests'])

In [17]:
# Let's check if the weather is good for our planned activities!
# This is like making sure you don't plan a picnic on a rainy day or a beach trip during a storm.

ACTIVITY_AND_WEATHER_ARE_COMPATIBLE_SYSTEM_PROMPT = """
You are a Weather Compatibility Analyst for AgentsVille's tourism department. Your job is to determine if an activity is compatible with the current weather conditions.

## Task
Analyze the activity description and weather condition to determine if they are compatible. Consider:
1. Is the activity primarily indoor or outdoor?
2. Does the activity description mention weather contingencies or backup plans?
3. Would the weather condition significantly impact the enjoyment or safety of the activity?

When there is not enough information, assume the activity IS_COMPATIBLE with the weather. Also, look out for backup options mentioned in the activity description.

## Output format

    REASONING:
    First, I'll determine if this is an indoor or outdoor activity.
    Then, I'll check if there are any weather contingencies mentioned.
    Finally, I'll assess if the current weather would negatively impact this activity.

    FINAL ANSWER:
    [IS_COMPATIBLE, IS_INCOMPATIBLE]

## Examples

Example 1:
Activity: Outdoor Yoga in the Park
Description: Join us for a rejuvenating yoga session in Central Park. This is a fully outdoor event with no shelter.
Weather Condition: rainy

REASONING:
This is clearly an outdoor activity with no mention of indoor alternatives or rain contingencies. Yoga on wet ground in the rain would be uncomfortable and potentially unsafe. The rainy weather would significantly impact the enjoyment and practicality of this activity.

FINAL ANSWER:
IS_INCOMPATIBLE

Example 2:
Activity: Museum Tour
Description: Explore our city's history through this guided tour of the National Museum. The entire event takes place inside our climate-controlled building.
Weather Condition: thunderstorm

REASONING:
This is an indoor activity that takes place entirely within a building. The weather outside would not affect the experience of touring the museum. The thunderstorm has no impact on this activity.

FINAL ANSWER:
IS_COMPATIBLE

Example 3:
Activity: Garden Festival
Description: Celebrate spring in our botanical gardens. In case of rain, activities will move to our large greenhouse and indoor pavilion.
Weather Condition: rainy

REASONING:
While this is primarily an outdoor event, the description specifically mentions a backup plan for rain. Activities will be moved to the greenhouse and indoor pavilion, allowing the event to continue despite the weather.

FINAL ANSWER:
IS_COMPATIBLE
""".strip()


def eval_activities_and_weather_are_compatible(
        vacation_info: VacationInfo, final_output: TravelPlan
):
    """Verifies that no outdoor-only activities are scheduled during inclement weather conditions.

    Args:
        vacation_info (dict): Contains the vacation details
        final_output (dict): Contains the itinerary details including daily activities and weather conditions

    Raises:
        AgentError: If any outdoor activities are scheduled during weather conditions that could ruin them
    """
    from project_lib import do_chat_completion

    activities_that_are_incompatible = []

    for itinerary_day in final_output.itinerary_days:
        weather_condition = itinerary_day.weather.condition

        for activity_recommendation in itinerary_day.activity_recommendations:
            resp = do_chat_completion(
                messages=[
                    {
                        "role": "system",
                        "content": ACTIVITY_AND_WEATHER_ARE_COMPATIBLE_SYSTEM_PROMPT,
                    },
                    {
                        "role": "user",
                        "content": f"Activity: {activity_recommendation.activity.name}\nDescription: {activity_recommendation.activity.description}\nWeather Condition: {weather_condition}",
                    },
                ],
                client=client,
                # This is a high-frequency use case, so we use a fast and cheap model.
                model=OpenAIModel.GPT_41_NANO,
            )

            if "IS_COMPATIBLE" in (resp or ""):
                is_compatible = True
            elif "IS_INCOMPATIBLE" in (resp or ""):
                is_compatible = False
            else:
                # Default to compatible if the response is unclear
                is_compatible = True

            if not is_compatible:
                activities_that_are_incompatible.append(
                    f"{activity_recommendation.activity.name} on {itinerary_day.date} during {weather_condition} weather"
                )

    if activities_that_are_incompatible:
        raise AgentError(
            f"The following activities are incompatible with the weather: {activities_that_are_incompatible}"
        )


In [18]:
# Define the tools that will be used by the ItineraryRevisionAgent
# These tools will help the agent refine the itinerary based on feedback and evaluation results.

import numexpr


def calculator_tool(expression: str) -> float:
    """Evaluates a mathematical expression and returns the result.

    Args:
        expression (str): A mathematical expression to evaluate (e.g., "2 + 2", "15 * 3", "50 - 10").

    Returns:
        float: The result of evaluating the expression.

    Example:
        >>> calculator_tool("2 + 2")
        4.0
    """
    try:
        # Use numexpr for safe evaluation of mathematical expressions
        result = float(numexpr.evaluate(expression))
        return result
    except Exception as e:
        return f"Error evaluating expression: {str(e)}"


def get_activities_by_date_tool(date: str, city: str) -> List[dict]:
    """Retrieves a list of available activities for a specific date and city.

    Args:
        date (str): The date to retrieve activities for, in YYYY-MM-DD format.
        city (str): The city to retrieve activities for.

    Returns:
        List[dict]: A list of activities available on the specified date in the specified city,
                   each validated against the Activity model and converted to a dictionary.

    Example:
        >>> get_activities_by_date_tool("2025-06-10", "AgentsVille")
        [{'activity_id': 'event-2025-06-10-0', 'name': 'FutureTech Breakfast Meet-Up', ...}]
    """
    from project_lib import call_activities_api_mocked
    resp = call_activities_api_mocked(date=date, city=city)

    # Convert the activities to a list of dictionaries that match the Activity model
    activities = []
    for activity_data in resp:
        try:
            # Validate the activity data against the Activity model
            activity = Activity(**activity_data)
            activities.append(activity.model_dump())
        except Exception as e:
            print(f"Error validating activity: {str(e)}")

    return activities


def run_evals_tool(travel_plan: dict) -> dict:
    """Evaluates a travel plan against a set of criteria and returns the evaluation results.

    Args:
        travel_plan (dict): A dictionary representing a travel plan, which should match the TravelPlan model.

    Returns:
        dict: A dictionary containing the evaluation results, including success status and any failures.

    Example:
        >>> run_evals_tool({"city": "AgentsVille", "start_date": "2025-06-10", ...})
        {"success": True, "failures": [], "eval_functions": ["eval_start_end_dates_match", ...]}
    """
    try:
        # Convert the travel_plan dictionary to a TravelPlan object
        travel_plan_obj = TravelPlan.model_validate(travel_plan)

        # Run the evaluation functions
        eval_results = get_eval_results(
            vacation_info=vacation_info,
            final_output=travel_plan_obj,
            eval_functions=[
                eval_start_end_dates_match,
                eval_total_cost_is_accurate,
                eval_total_cost_is_within_budget,
                eval_itinerary_events_match_actual_events,
                eval_itinerary_satisfies_interests,
                eval_activities_and_weather_are_compatible
            ]
        )

        # Return the evaluation results as a dictionary
        return eval_results.model_dump()
    except Exception as e:
        return {"success": False, "failures": [str(e)], "eval_functions": []}


def final_answer_tool(travel_plan: dict) -> dict:
    """Provides the final answer in a structured format.

    This tool should be called after the run_evals_tool to provide the final travel plan.

    Args:
        travel_plan (dict): A dictionary representing a travel plan, which should match the TravelPlan model.

    Returns:
        dict: The validated travel plan.

    Example:
        >>> final_answer_tool({"city": "AgentsVille", "start_date": "2025-06-10", ...})
        {"city": "AgentsVille", "start_date": "2025-06-10", ...}
    """
    try:
        # Convert the travel_plan dictionary to a TravelPlan object to validate it
        travel_plan_obj = TravelPlan.model_validate(travel_plan)

        # Return the validated travel plan as a dictionary
        return travel_plan_obj.model_dump()
    except Exception as e:
        return {"error": str(e)}


# Define a function to get tool descriptions as a string
def get_tool_descriptions_string(tools_dict):
    """Returns a string containing descriptions of the available tools.

    Args:
        tools_dict (dict): A dictionary mapping tool names to tool functions.

    Returns:
        str: A formatted string containing tool descriptions.
    """
    tool_descriptions = []

    for tool_name, tool_func in tools_dict.items():
        # Get the docstring and signature of the tool function
        docstring = tool_func.__doc__ or "No description available."
        import inspect
        signature = str(inspect.signature(tool_func))

        # Format the tool description
        tool_description = f"Tool: {tool_name}{signature}\n{docstring}\n"
        tool_descriptions.append(tool_description)

    return "\n".join(tool_descriptions)


# Define the available tools
ALL_TOOLS = {
    "calculator_tool": calculator_tool,
    "get_activities_by_date_tool": get_activities_by_date_tool,
    "run_evals_tool": run_evals_tool,
    "final_answer_tool": final_answer_tool
}

# Define the traveler feedback
TRAVELER_FEEDBACK = "We would like to have at least 2 activities per day to make the most of our trip."

# Define the system prompt for the ItineraryRevisionAgent
ITINERARY_REVISION_AGENT_SYSTEM_PROMPT = f"""
You are an Itinerary Revision Agent for AgentsVille. Your role is to refine and improve travel plans based on feedback and evaluation results.

## Task

Revise the provided itinerary based on the following requirements:
1. Incorporate the traveler's feedback to improve the itinerary
2. Ensure there are at least 2 activities per day
3. Verify that activities are compatible with the weather conditions
4. Make sure the total cost stays within the budget
5. Use the available tools to evaluate and improve the itinerary
6. Before finalizing, run the evaluation tools again to ensure all criteria are met

## Available Tools

{get_tool_descriptions_string(ALL_TOOLS)}

## ReAct Cycle

You will follow the THINK-ACT-OBSERVE cycle:

1. THINK: Analyze the current situation, consider options, and decide what to do next.
2. ACT: Call a tool using the specified JSON format.
3. OBSERVE: Review the result of the tool call.
4. Repeat until you're ready to provide the final answer.

You must run the run_evals_tool before using the final_answer_tool to ensure the itinerary meets all criteria.

After running the run_evals_tool and confirming that all criteria are met (or making necessary adjustments if they are not), you MUST use the final_answer_tool to provide the final answer. This is critical - your task is not complete until you call the final_answer_tool with a valid travel plan.

The final_answer_tool expects a complete travel plan that follows the TravelPlan model structure. Make sure your final travel plan includes all required fields and meets all the requirements specified in the task.

## Output Format

    THOUGHT:
    I need to analyze the current itinerary and identify areas for improvement.
    I should consider the traveler's feedback, weather conditions, budget constraints, and activity distribution.
    Then I'll use the appropriate tools to make and verify my changes.

    ACTION:
    {{
      "tool_name": "tool_name",
      "arguments": {{
        "arg1": "value1",
        "arg2": "value2"
      }}
    }}

## Context

Traveler Feedback: "{TRAVELER_FEEDBACK}"

Vacation Information:
{vacation_info.model_dump_json(indent=2)}

Weather Forecast:
{json.dumps(weather_for_dates, indent=2)}

Pydantic Models:
- TravelPlan: Contains city, start_date, end_date, total_cost, and itinerary_days
- ItineraryDay: Contains date, weather, and activity_recommendations
- ActivityRecommendation: Contains activity and reasons_for_recommendation
- Activity: Contains activity_id, name, start_time, end_time, location, description, price, and related_interests
- Weather: Contains temperature, temperature_unit, and condition
"""  # noqa


# Define the ItineraryRevisionAgent class
class ItineraryRevisionAgent(ChatAgent):
    """An agent that revises itineraries based on feedback and evaluation results."""
    system_prompt = ITINERARY_REVISION_AGENT_SYSTEM_PROMPT

    def __init__(self, client=None, model=None):
        super().__init__(client=client, model=model)
        self.tools = ALL_TOOLS

    def revise_itinerary(self, initial_itinerary, model="gpt-3.5-turbo"):
        """
        Revise the itinerary using the ReAct cycle.

        Args:
            initial_itinerary: A TravelPlan object containing the initial itinerary
            model: The model to use for revision

        Returns:
            A revised TravelPlan object
        """
        # Reset the agent
        self.reset()
        self.model = model

        # Convert the initial itinerary to a string
        initial_itinerary_str = initial_itinerary.model_dump_json(indent=2)

        # Set up the system prompt with more specific guidance
        self.system_prompt = f"""You are a helpful AI assistant specializing in revising travel itineraries.
    Your goal is to improve an initial travel plan by checking for issues and making corrections.

    Follow these steps:
    1. Analyze the initial itinerary for issues
    2. Check weather conditions for outdoor activities
    3. Verify budget constraints are met
    4. Ensure activities match traveler interests
    5. Improve the schedule for better flow and experience
    6. When you're satisfied with the revisions, provide the FINAL ANSWER in valid JSON format

    Here are the tools you can use:
    {get_tool_descriptions_string(self.tools)}

    IMPORTANT: Once you've completed your revisions, clearly indicate your FINAL ANSWER with the 
    complete revised itinerary in valid JSON format that matches the TravelPlan schema.
    """

        # Set the user message
        user_message = f"""Please revise this travel itinerary. Analyze it for issues and improve it.
    Here is the initial itinerary:

    {initial_itinerary_str}

    Think step by step, use tools as needed, and provide a revised itinerary as your final answer.
    """

        # Add the user message
        self.add_message("user", user_message)

        # Set up variables for the ReAct cycle
        max_iterations = 15  # Increased from 10 to 15
        current_iteration = 0
        final_answer = None

        # Start the ReAct cycle
        while current_iteration < max_iterations and final_answer is None:
            current_iteration += 1

            # Get the assistant's response
            response = self.get_response()

            # Check if the response contains a final answer
            if "FINAL ANSWER" in response:
                # Extract the final answer
                final_answer = response
                break

            # Parse the response to extract thought and action
            thought = None
            action = None

            # Extract thought if present
            if "THOUGHT:" in response:
                thought_parts = response.split("THOUGHT:")
                if len(thought_parts) > 1:
                    if "ACTION:" in thought_parts[1]:
                        thought = thought_parts[1].split("ACTION:")[0].strip()
                    else:
                        thought = thought_parts[1].strip()

            # Extract action if present
            if "ACTION:" in response:
                action_parts = response.split("ACTION:")
                if len(action_parts) > 1:
                    action = action_parts[1].strip()

            # Add thought to messages if present
            if thought:
                self.add_message("assistant", f"THOUGHT: {thought}")

            # Process action if present
            if action:
                try:
                    # Clean up the action to make it valid JSON
                    action = action.strip()
                    if action.endswith('```'):
                        action = action.split('```')[0].strip()

                    # Try to extract JSON from the action if it's not already valid JSON
                    if not action.startswith("{"):
                        import re
                        json_match = re.search(r'({.*})', action, re.DOTALL)
                        if json_match:
                            action = json_match.group(1)

                    # Parse the action JSON
                    import json
                    from json_repair import repair_json

                    # Try to parse the action as JSON
                    try:
                        action_json = json.loads(action)
                    except json.JSONDecodeError:
                        # If parsing fails, try to repair the JSON
                        try:
                            repaired_action = repair_json(action)
                            action_json = json.loads(repaired_action)
                            print(f"Repaired JSON: {repaired_action}")
                        except Exception as repair_error:
                            # If repair fails, try a more aggressive approach
                            print(f"JSON repair failed: {str(repair_error)}")
                            # Look for tool_name and arguments in the action
                            import re
                            tool_name_match = re.search(r'"tool_name"\s*:\s*"([^"]+)"', action)
                            tool_name = tool_name_match.group(1) if tool_name_match else None

                            # If we found a tool_name, try to extract arguments
                            if tool_name and tool_name == "final_answer_tool":
                                # For final_answer_tool, try to extract the travel plan
                                arguments_match = re.search(r'"arguments"\s*:\s*({.*})', action, re.DOTALL)
                                if arguments_match:
                                    try:
                                        arguments_json = repair_json(arguments_match.group(1))
                                        arguments = json.loads(arguments_json)
                                    except:
                                        # If all else fails, create a minimal valid travel plan
                                        arguments = {"travel_plan": initial_itinerary.model_dump()}
                                else:
                                    arguments = {"travel_plan": initial_itinerary.model_dump()}

                                # Create a valid action_json
                                action_json = {"tool_name": "final_answer_tool", "arguments": arguments}
                            else:
                                # For other tools, just raise the error
                                raise

                    # Extract tool_name and arguments from the parsed JSON
                    tool_name = action_json.get("tool_name")
                    arguments = action_json.get("arguments", {})

                    # Check if the tool exists
                    if tool_name in self.tools:
                        # Execute the tool
                        tool_result = self.tools[tool_name](**arguments)

                        # If the tool is final_answer_tool, we're done
                        if tool_name == "final_answer_tool":
                            final_answer = tool_result
                            break

                        # Add the tool result as an observation
                        observation = f"OBSERVATION: {json.dumps(tool_result, indent=2)}"
                        self.add_message("user", observation)
                    else:
                        # Tool not found
                        observation = f"OBSERVATION: Error: Tool '{tool_name}' not found. Available tools: {list(self.tools.keys())}"
                        self.add_message("user", observation)
                except Exception as e:
                    # Error parsing or executing the action
                    observation = f"OBSERVATION: Error: {str(e)}"
                    self.add_message("user", observation)
            else:
                # No action found
                observation = "OBSERVATION: Error: No ACTION found in the response. Please provide an ACTION."
                self.add_message("user", observation)

        # If we didn't get a final answer after max_iterations, raise an exception
        if final_answer is None:
            raise Exception(f"Failed to get a final answer after {max_iterations} iterations.")

        # Convert the final answer to a TravelPlan object
        try:
            # Extract the JSON part from the final answer
            import re
            import json
            from json_repair import repair_json

            # Try to find JSON in the final answer
            json_match = re.search(r'```json\n([\s\S]*?)\n```', final_answer)
            if json_match:
                json_str = json_match.group(1)
            else:
                # Try to find any JSON-like structure
                json_str = re.search(r'{[\s\S]*}', final_answer).group(0)

            # Try to parse it, using json_repair if needed
            try:
                travel_plan_dict = json.loads(json_str)
            except json.JSONDecodeError:
                # If standard parsing fails, try to repair the JSON
                repaired_json = repair_json(json_str)
                travel_plan_dict = json.loads(repaired_json)

            # Convert to TravelPlan object
            from pydantic import parse_obj_as
            from typing import Dict, Any
            travel_plan = parse_obj_as(Dict[str, Any], travel_plan_dict)

            return travel_plan

        except Exception as e:
            print(f"Error parsing the final answer: {str(e)}")
            print("Final answer received:")
            print(final_answer)
            return None


# Create an instance of the ItineraryRevisionAgent
itinerary_revision_agent = ItineraryRevisionAgent(client=client, model=MODEL)



╔══════════════════════════════════════[ ItineraryRevisionAgent - System Prompt ]══════════════════════════════════════╗
║ You are an Itinerary Revision Agent for AgentsVille. Your role is to refine and improve travel plans based on        ║
║ feedback and evaluation results.                                                                                     ║
║ ## Task                                                                                                              ║
║ Revise the provided itinerary based on the following requirements:                                                   ║
║ 1. Incorporate the traveler's feedback to improve the itinerary                                                      ║
║ 2. Ensure there are at least 2 activities per day                                                                    ║
║ 3. Verify that activities are compatible with the weather conditions                                                 ║
║ 4. Make sure the total cost s

In [19]:
# Use the ItineraryRevisionAgent to revise the initial itinerary
# This will demonstrate the ReAct cycle and the use of tools to improve the itinerary

# Check if we have an initial itinerary
if travel_plan_1 is not None:
    # Revise the itinerary
    travel_plan_2 = itinerary_revision_agent.revise_itinerary(
        initial_itinerary=travel_plan_1,
        model=MODEL,  # Optionally, you can change the model here
    )

    if travel_plan_2 is not None:
        print("✅ Itinerary revised successfully. Congratulations!")

        # Compare the initial and revised itineraries
        print("\nInitial Itinerary:")
        print(f"Total Cost: {travel_plan_1.total_cost}")
        print(f"Number of Activities: {sum(len(day.activity_recommendations) for day in travel_plan_1.itinerary_days)}")

        print("\nRevised Itinerary:")
        print(f"Total Cost: {travel_plan_2.total_cost}")
        print(f"Number of Activities: {sum(len(day.activity_recommendations) for day in travel_plan_2.itinerary_days)}")

        # Check if the revised itinerary meets the requirements
        # 1. At least 2 activities per day
        days_with_less_than_2_activities = [
            day.date for day in travel_plan_2.itinerary_days
            if len(day.activity_recommendations) < 2
        ]

        if days_with_less_than_2_activities:
            print(f"❌ The following days still have less than 2 activities: {days_with_less_than_2_activities}")
        else:
            print("✅ All days have at least 2 activities.")

        # 2. Total cost is within budget
        if travel_plan_2.total_cost <= vacation_info.budget:
            print(f"✅ Total cost ({travel_plan_2.total_cost}) is within budget ({vacation_info.budget}).")
        else:
            print(f"❌ Total cost ({travel_plan_2.total_cost}) exceeds budget ({vacation_info.budget}).")

        # 3. Run the evaluation functions to check if the revised itinerary meets all criteria
        eval_results = get_eval_results(
            vacation_info=vacation_info,
            final_output=travel_plan_2,
            eval_functions=[
                eval_start_end_dates_match,
                eval_total_cost_is_accurate,
                eval_total_cost_is_within_budget,
                eval_itinerary_events_match_actual_events,
                eval_itinerary_satisfies_interests,
                eval_activities_and_weather_are_compatible
            ]
        )

        if eval_results.success:
            print("✅ The revised itinerary meets all criteria.")
        else:
            print("❌ The revised itinerary does not meet all criteria:")
            for failure in eval_results.failures:
                print(f"  - {failure}")
else:
    print("❌ No initial itinerary to revise.")



╔══════════════════════════════════════[ ItineraryRevisionAgent - System Prompt ]══════════════════════════════════════╗
║ You are an Itinerary Revision Agent for AgentsVille. Your role is to refine and improve travel plans based on        ║
║ feedback and evaluation results.                                                                                     ║
║ ## Task                                                                                                              ║
║ Revise the provided itinerary based on the following requirements:                                                   ║
║ 1. Incorporate the traveler's feedback to improve the itinerary                                                      ║
║ 2. Ensure there are at least 2 activities per day                                                                    ║
║ 3. Verify that activities are compatible with the weather conditions                                                 ║
║ 4. Make sure the total cost s